# Taking a New Approach: Hamiltonian Monte Carlo (HMC)

This notebook extends the origin discussion of Bayesian Inference that I worked for my FSharp Advent of Code - 2020 [submission](https://nbviewer.jupyter.org/github/MokoSan/FSharpAdvent_2020/blob/main/BayesianInferenceInF%23.ipynb). I have been meaning to get back to some non-Generative AI Data Science methodologies and figured I left where I started by improving on the deficiencies of the Symmetric Metropolis Hastings algorithm that was implemented in the previous post.

Furthermore, back in 2020, the ecosystem of .NET Interactive Notebooks was is its infancy - currently, Polyglot Notebooks, the successor to .NET Interactive Notebooks is a mature product with a lot more users and functionality such as importing notebooks. This improvement in the tooling has made this experience a lot smoother and the process to extend the behavior simpler.

## Acknowledgements

Special thanks to Navya Arora for her contribution with the data and helping me validate the overall idea. She was eager in helping and provided some great insights during the development phase of this algorithm.

## Deficiencies of Symmeteric Metropolis Hastings

Before diving into the Hamiltonian Monte Carlo algorithm, it's worth reiterate and elucidating on the deficiencies of our previous approach: the Symmeteric Metropolis Hastings algorithm and large number of which can be listed but the top 3, in my opinion working with this algorithm are:

### 1. Limited Exploration of Parameter Space:

The Symmetric Metropolis Hastings algorithm relies heavily on the proposal distribution. If the proposal distribution is poorly tuned (e.g., too narrow or too wide), the algorithm may fail to explore the parameter space effectively, leading to slow convergence or biased results.

### 2. High Dependence on Hyperparameters:

The algorithm requires careful tuning of hyperparameters like the proposal distribution's step size (delta). Poorly chosen values can result in either excessive rejection of proposals or inefficient exploration.

### 3. Autocorrelation in Chains:

The algorithm suffers from autocorrelation, where consecutive samples are highly dependent on each other. This reduces the effective sample size and makes the posterior approximation less reliable.

## An Introduction to Hamiltonian Monte Carlo

HMC combines ideas from physics and numerical optimization to simulate the posterior distribution. It uses the gradient of the log-posterior to guide the sampling process, reducing random walk behavior and improving convergence.

The two key components of this algorithm are __Momentum Sampling__ and __Leapfrog Integration__.

### Momentum Sampling

In HMC, the posterior distribution is treated as a physical system, where the parameter values (position) are associated with potential energy, and an auxiliary variable called momentum is introduced to simulate kinetic energy.

Momentum is sampled from a Gaussian distribution (e.g., Normal(0, 1)).
This auxiliary variable helps guide the exploration of the parameter space by simulating physical dynamics.

Momentum sampling allows the algorithm to escape local modes and explore the posterior more efficiently.
It reduces random walk behavior, which is common in simpler MCMC methods like Metropolis Hastings.
Mathematical Representation:

Momentum ( p ) is sampled as: $p \sim \mathcal{N}(0, M)$ where ( M ) is the mass matrix (often set to identity for simplicity).

### Leapfrog Integration

Leapfrog integration is a numerical method used to simulate Hamiltonian dynamics in HMC. It updates both the position (parameter values) and momentum iteratively, ensuring energy conservation and stability.  Leapfrog integration ensures that the Hamiltonian dynamics are simulated accurately, preserving the total energy of the system. It avoids numerical instability and ensures reversibility, which is crucial for the acceptance step in HMC.

#### Steps of :

**Half-Step Momentum Update**

The momentum is updated using the gradient of the log-posterior: 

$p_{\text{new}} = p_{\text{current}} + \frac{\epsilon}{2} \cdot \nabla \log p(\theta)$ where ( $\epsilon$ ) is the step size.

**Full-Step Position Update**

The position is updated using the new momentum: 

$\theta_{\text{new}} = \theta_{\text{current}} + \epsilon \cdot p_{\text{new}}$ 

**Second Half-Step Momentum Update**

The momentum is updated again using the gradient at the new position: 

$p_{\text{final}} = p_{\text{new}} + \frac{\epsilon}{2} \cdot \nabla \log p(\theta_{\text{new}})$ 

## All the Steps: 

1. **Initialization**: Start with an initial position (parameter value).
2. **Momentum Sampling**: Sample a momentum variable from a Gaussian distribution.
3. **Leapfrog Integration**: Simulate the Hamiltonian dynamics using the leapfrog method to propose a new position and momentum.
4. **Acceptance Step**: Accept or reject the proposed position based on the Hamiltonian.


In [1]:
#!import BayesianInferenceEngine.dib

Installed Packages MathNet.Numerics, 5.0.0 Newtonsoft.Json, 13.0.3 XPlot.Plotly, 4.1.0

In [2]:
open System
open MathNet.Numerics.Distributions

type HamiltonianMonteCarloRequest = 
    { StepSize          : float
      LeapfrogSteps     : int
      Iterations        : int
      BurnInIterationsPct : float
      Chains            : int }

type HamiltonianMonteCarloResult =
    { Chains         : seq<float seq>
      AcceptanceRate : float }

let computeGradient (model: SimpleBayesianNetworkModel) (position: float) : float =
    // Compute the gradient of the log-posterior at the given position.
    let priorGradient = model.GetPriorProbability position
    let likelihoodGradient = model.GetLikelihoodProbability position
    priorGradient + likelihoodGradient // Simplified for illustration

let leapfrogIntegration (position: float) (momentum: float) (stepSize: float) (model: SimpleBayesianNetworkModel) : float * float =
    // Perform leapfrog integration to propose new position and momentum.
    let gradient = computeGradient model position
    let newMomentum = momentum + (stepSize / 2.0) * gradient
    let newPosition = position + stepSize * newMomentum
    let updatedGradient = computeGradient model newPosition
    let finalMomentum = newMomentum + (stepSize / 2.0) * updatedGradient
    newPosition, finalMomentum

let runHamiltonianMonteCarlo (request: HamiltonianMonteCarloRequest) (model: SimpleBayesianNetworkModel) : HamiltonianMonteCarloResult =
    let zeroOneUniform = ContinuousUniform()
    let mutable acceptanceCount = 0
    let chains = 
        seq { 1 .. request.Chains }
        |> Seq.map (fun _ ->
            let mutable position = 0.0 // Initialize position
            let mutable chain = []
            for _ in 1 .. request.Iterations do
                let momentum = Normal(0.0, 1.0).Sample() // Sample momentum
                let proposedPosition, proposedMomentum = leapfrogIntegration position momentum request.StepSize model
                let currentHamiltonian = -model.GetPosteriorWithoutScalingFactor position + (momentum ** 2.0) / 2.0
                let proposedHamiltonian = -model.GetPosteriorWithoutScalingFactor proposedPosition + (proposedMomentum ** 2.0) / 2.0
                let acceptanceRatio = Math.Exp(currentHamiltonian - proposedHamiltonian)
                if zeroOneUniform.Sample() < acceptanceRatio then
                    position <- proposedPosition
                    acceptanceCount <- acceptanceCount + 1
                chain <- chain @ [position]
            chain |> Seq.skip (int (request.BurnInIterationsPct / 100.0 * float request.Iterations))
        )
    let acceptanceRate = float acceptanceCount / float (request.Iterations * request.Chains)
    { Chains = chains; AcceptanceRate = acceptanceRate }

### Example: Normal Prior and Normal Likelihood

In [3]:
let model = @"x ~ Normal(μ,τ) 
              y|x ~ Normal(x,σ) : observed"
let parsedModel = parseModel model
let paramList = "{Parameters: {μ : 5, τ : 3.1622, σ : 1}, observed : [9.37,10.18,9.16,11.60,10.33]}"
let simpleModel = 
    SimpleBayesianNetworkModel.Construct "Normal-Normal" parsedModel (deserializeParameters paramList)

let hmcRequest : HamiltonianMonteCarloRequest = 
    { StepSize          = 0.01
      LeapfrogSteps     = 10
      Iterations        = 50_000 // Sparingly choose this.
      BurnInIterationsPct = 10.0
      Chains            = 4 }

let hmcResult = runHamiltonianMonteCarlo hmcRequest simpleModel

// Visualize the first chain
let firstChain = Seq.head hmcResult.Chains

Histogram(x = firstChain)
|> Chart.Plot
|> Chart.WithTitle "Normal Prior and Normal Likelihood"
|> Chart.WithWidth 700
|> Chart.WithHeight 500

Height,500
Id,0a8d304b-c017-4ee4-832d-dbb193759469
PlotlySrc,https://cdn.plot.ly/plotly-latest.min.js
Width,700


![Ham](./Images/HMC_NN.png)

In [ ]:
let model = @"x ~ Normal(μ,τ) 
              y|x ~ Normal(x,σ) : observed"
let parsedModel = parseModel model
let paramList = "{Parameters: {μ : 5, τ : 6, σ : 1}, observed : [9.37,10.18,9.16,11.60,10.33]}"
let simpleModel = 
    SimpleBayesianNetworkModel.Construct "Normal-Normal" parsedModel (deserializeParameters paramList)

let hmcRequest : HamiltonianMonteCarloRequest = 
    { StepSize          = 0.01
      LeapfrogSteps     = 10
      Iterations        = 50_000 // Sparingly choose this.
      BurnInIterationsPct = 10.0
      Chains            = 4 }

let hmcResult = runHamiltonianMonteCarlo hmcRequest simpleModel

// Visualize the first chain
let firstChain = Seq.head hmcResult.Chains

Histogram(x = firstChain)
|> Chart.Plot
|> Chart.WithTitle "Normal Prior and Normal Likelihood"
|> Chart.WithWidth 700
|> Chart.WithHeight 500

## Disadvantages of HMC

The HMC algorithm is not without its flaws and some of which are:

### 1. Computational Complexity

- Gradient Computation:
HMC requires the computation of the gradient of the log-posterior at every step. For complex models, this can be computationally expensive, especially when the posterior involves high-dimensional parameter spaces or intricate likelihood functions.

- Leapfrog Integration:
The iterative nature of leapfrog integration adds additional computational overhead compared to simpler MCMC methods like Metropolis Hastings.

### 2. Sensitivity to Hyperparameters

- Step Size $\epsilon$:
The step size must be carefully tuned. If it is too small, the algorithm takes many small steps, increasing runtime. If it is too large, the leapfrog integration becomes unstable, leading to poor sampling.

- Number of Leapfrog Steps:
The number of leapfrog steps must also be tuned. Too few steps result in insufficient exploration, while too many steps increase computational cost without significant improvement in sampling.

### 3. Difficulty in High-Dimensional Spaces

- Mass Matrix:
In high-dimensional parameter spaces, the choice of the mass matrix (used in momentum sampling) becomes critical. Using a simple identity matrix may lead to inefficient exploration, while estimating a more complex mass matrix adds computational overhead.

- Curvature:
HMC struggles with posterior distributions that have regions of varying curvature (e.g., sharp peaks and flat valleys). This can lead to inefficient sampling or instability.

## Conclusion

Hamiltonian Monte Carlo (HMC) represents a significant advancement in Bayesian inference, offering a powerful alternative to traditional MCMC methods like Metropolis-Hastings. By leveraging concepts from physics—specifically, Hamiltonian dynamics—HMC enables more efficient exploration of complex posterior landscapes, reducing autocorrelation and improving convergence rates. While it introduces additional computational complexity and requires careful tuning of hyperparameters, the benefits in terms of sampling quality and scalability are substantial, especially for high-dimensional models.

Through practical examples, we've seen how HMC can be applied to both synthetic and real-world data, providing robust posterior estimates even in challenging scenarios. As probabilistic modeling continues to grow in importance across scientific and industrial domains, mastering advanced inference techniques like HMC will be invaluable for practitioners seeking both accuracy and efficiency in their analyses.
